# Initialisation

In [ ]:
from __future__ import annotations

from pathlib import Path

import mqt.pathfinder.utils as utils
from mqt.pathfinder import cost_functions as cf
from mqt.pathfinder.graph import Graph

In [ ]:
with Path("input/graph").open() as file:
    graph = Graph.read(file)
graph.plot()

# QUBO Construction

### Parameters

In [ ]:
encoding_type = cf.EncodingType.UNARY
n_paths = 1
max_path_length = graph.n_vertices
loop = True

In [ ]:
settings = cf.PathFindingQUBOGeneratorSettings(encoding_type, n_paths, max_path_length, loop)

generator = cf.PathFindingQUBOGenerator(
    # objective_function=cf.MinimisePathLength(path_ids=[1]), graph=graph, settings=settings
    objective_function=None,
    graph=graph,
    settings=settings,
)

### Constraints

In [ ]:
generator.add_constraint(cf.PathIsValid(path_ids=[1]))
generator.add_constraint(cf.PathContainsVerticesExactlyOnce(vertex_ids=graph.all_vertices, possible_paths=[1]))

### Generate QUBO Formulation

In [ ]:
print(generator._select_lambdas())
generator.construct()

In [ ]:
generator.construct_expansion()

In [ ]:
A = generator.construct_qubo_matrix()
utils.print_matrix(A)

# Test Results

### Brute Force Optimisation

In [ ]:
import numpy as np

(best_test, best_score) = utils.optimise_classically(A)

x = np.array(best_test)
pth = generator.decode_bit_array(best_test)
print(pth)

### Operator: Classical Eigensolver

In [ ]:
from typing import TYPE_CHECKING

import numpy as np
import numpy.typing as npt
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit.result import QuasiDistribution

if TYPE_CHECKING:
    from qiskit.quantum_info import Statevector


def bitfield(n: int, L: int) -> list[int]:
    result = np.binary_repr(n, L)
    return [int(digit) for digit in result]


def sample_most_likely(
    state_vector: QuasiDistribution | Statevector | dict[str, float]
) -> npt.NDArray[np.int_ | np.float64]:
    """Compute the most likely binary string from state vector.
    Args:
        state_vector: State vector or quasi-distribution.

    Returns:
        Binary string as an array of ints.
    """
    values = (
        list(state_vector.values())
        if isinstance(state_vector, QuasiDistribution)
        else [state_vector[key] for key in state_vector]
        if isinstance(state_vector, dict)
        else state_vector
    )
    n = int(np.log2(len(values)))
    k = np.argmax(np.abs(values))
    x = bitfield(k, n)
    x.reverse()
    return np.asarray(x)


op = generator.construct_operator()

npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(op)
x = sample_most_likely(result.eigenstate)
print(x)
print(generator.decode_bit_array(x))
print(result.eigenvalue)

## Quantum Circuits

### QAOA

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit.utils import algorithm_globals

seed = 100000  # random.randint(10000, 20000)
algorithm_globals.random_seed = seed
op = generator.construct_operator()

sampler = Sampler()
optimizer = COBYLA()
qaoa = QAOA(sampler, optimizer, reps=3)

result = qaoa.compute_minimum_eigenvalue(op)
x = sample_most_likely(result.eigenstate)
print(generator.decode_bit_array(x))
print(result.eigenvalue)

### VQE

In [ ]:
from qiskit import Aer
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import Sampler
from qiskit.utils import algorithm_globals


def sample_most_likely_dict(states: dict[str, float]) -> npt.NDArray[np.int_ | np.float64]:
    """Compute the most likely binary string from state vector.
    Args:
        state_vector: State vector or quasi-distribution.

    Returns:
        Binary string as an array of ints.
    """
    len(next(iter(states.keys())))
    x = max([(key, states[key]) for key in states], key=lambda e: e[1])[0]
    x = list(reversed([int(y) for y in x]))
    print(x)
    return np.asarray(x)


seed = 200000  # random.randint(10000, 20000)
algorithm_globals.random_seed = seed
op = generator.construct_operator()

# ansatz = TwoLocal(op.num_qubits, 'ry', 'cz', reps=2, entanglement='linear')
ansatz = EfficientSU2(op.num_qubits, reps=1)
# ansatz = TwoLocal(op.num_qubits, 'ry', 'cz', reps=2, entanglement='linear')
vqe = VQE(ansatz, optimizer, quantum_instance=Aer.get_backend("qasm_simulator"))

result = vqe.compute_minimum_eigenvalue(op)

x = sample_most_likely_dict(result.eigenstate)
print(generator.decode_bit_array(x))
print(result.eigenvalue)

# Other Problems

_Also define the starting vertex of the path_

In [ ]:
generator_new = cf.PathFindingQUBOGenerator(cf.MinimisePathLength([1], loop=True), graph, settings)
generator_new.add_constraint(cf.PathIsValid([1]))
generator_new.add_constraint(cf.PathIsLoop([1]))
generator_new.add_constraint(cf.PathContainsVerticesExactlyOnce(graph.all_vertices, [1]))


generator_new.add_constraint(cf.PathStartsAt([4], 1))


A = generator_new.construct_qubo_matrix()
(best_test, best_score) = utils.optimise_classically(A)

pth = generator_new.decode_bit_array(best_test)
print(pth)

_Find the shortest paths $\pi_1$ and $\pi_2$ from $s_p$ to $t_p$ respectively that don't interesect_

In [ ]:
(s1, t1) = 1, 5
(s2, t2) = 2, 6

settings = cf.PathFindingQUBOGeneratorSettings(encoding_type, 2, max_path_length)
generator_new = cf.PathFindingQUBOGenerator(cf.MinimisePathLength([1, 2]), graph, settings)
generator_new.add_constraint(cf.PathIsValid([1, 2]))
generator_new.add_constraint(cf.PathIsLoop([1, 2]))
generator_new.add_constraint(cf.PathStartsAt([s1], 1))
generator_new.add_constraint(cf.PathStartsAt([s2], 2))
generator_new.add_constraint(cf.PathEndsAt([t1], 1))
generator_new.add_constraint(cf.PathEndsAt([t2], 2))
generator_new.add_constraint(cf.PathsShareNoVertices([1, 2]))